<a href="https://colab.research.google.com/github/kaimakov-rk/kokoko/blob/master/Copy_of_%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_5_%D0%A1%D0%90%D0%9C%D0%9E%D0%95_%D0%9D%D0%9E%D0%92%D0%9E%D0%95_%D0%94%D0%9B%D0%AF_%D0%9F%D0%A0%D0%9E%D0%92%D0%95%D0%A0%D0%9A%D0%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Линейная регрессия

##Выбор и исключение переменных

Загрузите набор данных из файла ```Regressions datasets``` в соответствии с вашим вариантом (о смене датасета - индивидуально)

In [177]:
!pip install pandas
!pip install scikit-learn
!pip install matplotlib
!pip install numpy

In [178]:
import pandas as pd

df = pd.read_csv("/content/regression.csv")

### Предварительный анализ

1. Выбрать целевую переменную Y (Y - числоая переменная для задачи регрессии)
2. С помощью EDA (см практика) отобрать признаки, которые влияют на Y (можно определить гарфически, можно числами).
3. Обработать пропуски, при необходимости почистить от выбросов (тут важное замечание - не все выбросы это плохо)
4. Обосновать выбор переменных для исследования (корреляция, различия по классам, и так далее)

P.S. Важное замечание, слишком уникальные признаки (имена людей, названия город, ID юзера или чего-то еще - не берем, так как они не влияют на целевую переменную, а если и влияет, то это случайно)

In [179]:
df.head(25)

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0
5,6,2010,1,1,5,NaN,-19,-10.0,1017.0,NW,16.10,0,0
6,7,2010,1,1,6,NaN,-19,-9.0,1017.0,NW,19.23,0,0
7,8,2010,1,1,7,NaN,-19,-9.0,1017.0,NW,21.02,0,0
8,9,2010,1,1,8,NaN,-19,-9.0,1017.0,NW,24.15,0,0
9,10,2010,1,1,9,NaN,-20,-8.0,1017.0,NW,27.28,0,0


In [180]:
# 1. Target (´,,•ω•,,)♡

yname = "pm2.5"
Y = df[yname]
Y

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
43819     8.0
43820    10.0
43821    10.0
43822     8.0
43823    12.0
Name: pm2.5, Length: 43824, dtype: float64

In [181]:
# 2. Correlations ☆*:.｡.o(≧▽≦)o.｡.:*☆

correlation_matrix = df.drop(columns=[yname, 'No']).corrwith(Y, numeric_only=True)
correlation_matrix

year    -0.014690
month   -0.024069
day      0.082788
hour    -0.023116
DEWP     0.171423
TEMP    -0.090534
PRES    -0.047282
Iws     -0.247784
Is       0.019266
Ir      -0.051369
dtype: float64

Сильнее всего с целевым столбцом коррелируют столбцы DEWP и Iws (обратная корелляция). Корреляция довольно слабая, но тут уже ничего не поделать o(〒﹏〒)o

In [182]:
# 3. Обработка пропусков (・`ω´・)

df = df.dropna() # (T.N.D.) Total Null Deletion

4. Причины, по которым были выбраны столбцы DEWP и Iws: во первых, они сильнее всего коррелируют с целевой переменной. Во вторых, точка росы влияет на количество чего-то там в воздухе, т.к. пыль собирается в росе, а не разлетается по воздуху. Скорость ветра, вероятно, влияет на скорость распространения пыли по ветру. Вероятно, также на качество воздуха влияет и направление ветра.

In [183]:
# Я оказался прав (=⌒‿‿⌒=)

temp_df = df.copy()
temp_df["cbwd"] = temp_df["cbwd"].astype('category').cat.codes
temp_df[["cbwd"]].corrwith(Y, numeric_only=True)

cbwd    0.19715
dtype: float64

### Трансформация набора данных

(OneHotEncode для категориальных переменных)

(Standard or MinMax or бининг или другое преобразование для числовых)

P.S. Целевую переменную не преобразовываем, оставляем как есть


In [184]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Няшим категориальный признак (^=◕ᴥ◕=^)

#encoder = OneHotEncoder(handle_unknown='ignore')
#df[["cbwd"]] = encoder.fit_transform(df[["cbwd"]])

df[["cbwd"]] = df[["cbwd"]].astype('category')
dummies = pd.get_dummies(df["cbwd"])
df = pd.concat([df.drop(columns=["cbwd"]), dummies], axis=1)

In [185]:
df.head(5)

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,NE,NW,SE,cv
24,25,2010,1,2,0,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0
25,26,2010,1,2,1,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0
26,27,2010,1,2,2,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0
27,28,2010,1,2,3,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0
28,29,2010,1,2,4,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0


In [186]:
standard_scaler = StandardScaler()

numeric_cols = ['DEWP', 'Iws']

# Няшим простые численные признаки (=^･ｪ･^=)

df[numeric_cols] = standard_scaler.fit_transform(df[numeric_cols])

In [187]:
df[numeric_cols].describe()

,DEWP,Iws
count,4.175700e+04,4.175700e+04
mean,3.539356e-17,1.905807e-17
std,1.000012e+00,1.000012e+00
min,-2.892591e+00,-4.719510e-01
25%,-8.140912e-01,-4.449441e-01
50%,1.730880e-02,-3.727913e-01
75%,9.179922e-01,-3.943711e-02
max,1.818676e+00,1.091610e+01


### Разделение данных

Для начала, разделить данные на X и Y

Разбить данные на тестовую и обучающую выборки

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [188]:
# Невозможно приготовить нейросеть, не разбив данных ⊂(´• ω •`⊂)

from sklearn.model_selection import train_test_split

# С категориальной переменной получается ЕЩЁ ХУЖЕ
xnames = [#"NE",
          #"NW",
          #"SE",
          #"cv",
          "DEWP",
          "Iws"]

X = df[xnames]
y = df[[yname]].astype('float64')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Обучение
#### 1 Регрессия
- Обучить линейную регрессию на тренировочных данных (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)
- Сдлеать .predict() для X_train, X_test
- Рассчитать метрики качества MAE, MSE, MAPE, R2
- Вывести и написать выводы о коэфициентах регрессии
- Написать формулу регрессии

Сделать выводы о переобучении модели (см метрики на тестовой выборке) сделать выводы о метриках на тестовом наборе

#### 2 Ближайшие соседи
- Обучить KNN (https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor)
- Сдлеать .predict() для X_train, X_test
- Рассчитать метрики качества MAE, MSE, MAPE, R2

Сделать выводы о переобучении модели (см метрики на тестовой выборке) сделать выводы о метриках на тестовом наборе

In [189]:
# Во-первых

"""
⢸⠉⣹⠋⠉⢉⡟⢩⢋⠋⣽⡻⠭⢽⢉⠯⠭⠭⠭⢽⡍⢹⡍⠙⣯⠉⠉⠉⠉⠉⣿⢫⠉⠉⠉⢉⡟⠉⢿⢹⠉⢉⣉⢿⡝⡉⢩⢿⣻⢍⠉⠉⠩⢹⣟⡏⠉⠹⡉⢻⡍⡇
⢸⢠⢹⠀⠀⢸⠁⣼⠀⣼⡝⠀⠀⢸⠘⠀⠀⠀⠀⠈⢿⠀⡟⡄⠹⣣⠀⠀⠐⠀⢸⡘⡄⣤⠀⡼⠁⠀⢺⡘⠉⠀⠀⠀⠫⣪⣌⡌⢳⡻⣦⠀⠀⢃⡽⡼⡀⠀⢣⢸⠸⡇
⢸⡸⢸⠀⠀⣿⠀⣇⢠⡿⠀⠀⠀⠸⡇⠀⠀⠀⠀⠀⠘⢇⠸⠘⡀⠻⣇⠀⠀⠄⠀⡇⢣⢛⠀⡇⠀⠀⣸⠇⠀⠀⠀⠀⠀⠘⠄⢻⡀⠻⣻⣧⠀⠀⠃⢧⡇⠀⢸⢸⡇⡇
⢸⡇⢸⣠⠀⣿⢠⣿⡾⠁⠀⢀⡀⠤⢇⣀⣐⣀⠀⠤⢀⠈⠢⡡⡈⢦⡙⣷⡀⠀⠀⢿⠈⢻⣡⠁⠀⢀⠏⠀⠀⠀⢀⠀⠄⣀⣐⣀⣙⠢⡌⣻⣷⡀⢹⢸⡅⠀⢸⠸⡇⡇
⢸⡇⢸⣟⠀⢿⢸⡿⠀⣀⣶⣷⣾⡿⠿⣿⣿⣿⣿⣿⣶⣬⡀⠐⠰⣄⠙⠪⣻⣦⡀⠘⣧⠀⠙⠄⠀⠀⠀⠀⠀⣨⣴⣾⣿⠿⣿⣿⣿⣿⣿⣶⣯⣿⣼⢼⡇⠀⢸⡇⡇⠇
⢸⢧⠀⣿⡅⢸⣼⡷⣾⣿⡟⠋⣿⠓⢲⣿⣿⣿⡟⠙⣿⠛⢯⡳⡀⠈⠓⠄⡈⠚⠿⣧⣌⢧⠀⠀⠀⠀⠀⣠⣺⠟⢫⡿⠓⢺⣿⣿⣿⠏⠙⣏⠛⣿⣿⣾⡇⢀⡿⢠⠀⡇
⢸⢸⠀⢹⣷⡀⢿⡁⠀⠻⣇⠀⣇⠀⠘⣿⣿⡿⠁⠐⣉⡀⠀⠁⠀⠀⠀⠀⠀⠀⠀⠀⠉⠓⠳⠄⠀⠀⠀⠀⠋⠀⠘⡇⠀⠸⣿⣿⠟⠀⢈⣉⢠⡿⠁⣼⠁⣼⠃⣼⠀⡇
⢸⠸⣀⠈⣯⢳⡘⣇⠀⠀⠈⡂⣜⣆⡀⠀⠀⢀⣀⡴⠇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢽⣆⣀⠀⠀⠀⣀⣜⠕⡊⠀⣸⠇⣼⡟⢠⠏⠀⡇
⢸⠀⡟⠀⢸⡆⢹⡜⡆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⠋⣾⡏⡇⡎⡇⠀⡇
⢸⠀⢃⡆⠀⢿⡄⠑⢽⣄⠀⠀⠀⢀⠂⠠⢁⠈⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠀⠄⡐⢀⠂⠀⠀⣠⣮⡟⢹⣯⣸⣱⠁⠀⡇
⠈⠉⠉⠉⠉⠉⠉⠉⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⠉⠉⠉⠉⠉⠉⠁
"""

from sklearn.linear_model import LinearRegression

linreg = LinearRegression().fit(X_train, y_train)

In [190]:
train_prediction = linreg.predict(X_train)
test_prediction = linreg.predict(X_test)

In [191]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

def compute_metrics(y, predicted):
  # Оценка качества модели
  mae = mean_absolute_error(y, predicted)
  mse = mean_squared_error(y, predicted)
  r2 = r2_score(y, predicted)
  mape = mean_absolute_percentage_error(y, predicted)

  return {
      'MAE': mae,
      'MSE': mse,
      'R2': r2,
      'mape (reverted - % верно угаданых)': (1 - mape) * 100,
      'mape (original - % ошибок)': mape * 100,
  }

In [192]:
compute_metrics(test_prediction, y_test)

{'MAE': 65.13421479945681,
 'MSE': 7937.6482837520225,
 'R2': -11.53831564808967,
 'mape (reverted - % верно угаданых)': 19.893154714344906,
 'mape (original - % ошибок)': 80.1068452856551}

In [194]:
compute_metrics(train_prediction, y_train)

{'MAE': 65.15989262592642,
 'MSE': 7825.435007460901,
 'R2': -11.63920311035263,
 'mape (reverted - % верно угаданых)': 21.8447846861766,
 'mape (original - % ошибок)': 78.1552153138234}

In [195]:
coefficients = linreg.coef_
coefficients

array([[ 10.14978607, -20.03742584]])

Коэффициент точки росы составляет 10.14978607, это положительное число, довольно большое по модулю, значит данная переменная оказывает сильное влияние на целевую переменную. Чем больше значение этой переменной, тем сильнее загрязнен воздух.

Коэффициент скорости ветра составляет почти -20, значит скорость ветра также оказывает сильное влияние на целевую переменную. Чем больше скорость ветра, тем меньше загрязнен воздух.

Сама модель почему-то плохо работает...

In [196]:
# Во-вторых

from sklearn.neighbors import KNeighborsRegressor

negressor = KNeighborsRegressor(n_neighbors=2).fit(X_train, y_train)

In [197]:
knn_test_prediction = negressor.predict(X_test)
knn_test_prediction

array([[ 37. ],
       [  9. ],
       [ 32.5],
       ...,
       [125.5],
       [ 50.5],
       [ 75.5]])

In [198]:
knn_train_prediction = negressor.predict(X_train)
knn_train_prediction

array([[ 74.5],
       [183.5],
       [  9. ],
       ...,
       [168. ],
       [ 93.5],
       [ 34. ]])

In [199]:
compute_metrics(knn_train_prediction, y_train)

{'MAE': 56.10004646674054,
 'MSE': 7166.532544590199,
 'R2': -0.4804883257947026,
 'mape (reverted - % верно угаданых)': 32.46152275596907,
 'mape (original - % ошибок)': 67.53847724403093}

In [200]:
compute_metrics(knn_test_prediction, y_test)

{'MAE': 68.01400580551524,
 'MSE': 9608.88965892598,
 'R2': -0.9646428337365558,
 'mape (reverted - % верно угаданых)': 4.635782287278644,
 'mape (original - % ошибок)': 95.36421771272136}

ВЫВОДЫ

Исходя из полученных метрик можно понять, что модель работает недостаточно хорошо.

In [201]:
from IPython.display import Image
Image(url='https://img10.reactor.cc/pics/post/-_--anime-gif-Anime-madoka-magica-1414291.gif')

##Задание для гениев*

Удачи 😈

В практике есть примеры реализации ```linear_regression``` и ```knn_regressor```, но есть нюанс... Сейчас ни linear_regression, ни knn_regressor не поддерживают сохранения состояния для последующего возможного использования.

Необходимо - реализовать два класса LinearRegression и KNNRegression (примеры обучения моделек есть). В этих классах обязатльно должны быть два метода
```
.fit(X: np.array, y: np.array) -> None - обучить модель и сохранить внутреннее состояние модели
.predict(X) -> np.array - предсказать y на основе X
```
При инициализации класса - передавать параметры модели в момент инициализации. Например:

```
model = LinearRegression(**params)

model.fit(X_train, y_train)
model.predict(X_test)
```

В качестве тестов можно использовать KNN и LinearRegression из sklearn, результат работы ваших объектов должен примерно совпадать с результатами работы объектов из sklearn

В результате работы сделать сравнение с теми же данными в sklearn

In [202]:
"""
Результат работы
"""

'\nРезультат работы\n'